# 01 · Adaptive β Estimator Field Manual

This notebook operationalizes the **WAMECU manifesto** by stress-testing online estimators on a drifting bias field.
We simulate a sinusoidally nudged random walk for β, stream draws, and compare an EWMA tracker against a per-outcome Kalman filter.
Random seeds are fixed to keep the experiment reproducible.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from wamecu import BetaDriftConfig, beta_drift, ewma_estimator, kalman_tracker, stream_draws

plt.style.use('seaborn-v0_8')
RNG = np.random.default_rng(1234)

## Configure the drifting β field

We combine a light random walk with a sinusoidal push on outcome 0 to mimic a mechanical wobble.

In [ ]:
n_outcomes = 5
n_steps = 360
config = BetaDriftConfig(
    n_steps=n_steps,
    n_outcomes=n_outcomes,
    walk_scale=0.025,
    sin_amplitude=0.06,
    sin_period=90,
    clip=0.9,
    seed=7,
)
true_beta = beta_drift(config)

# Stream draws driven by the drifting β field
stream_seed = 31415
observations = stream_draws(true_beta, seed=stream_seed)
assert len(observations) == n_steps

## Run online estimators

Both estimators only ingest the categorical stream and must track β in real time.

In [ ]:
ewma_history = ewma_estimator(observations, n_outcomes=n_outcomes, alpha=0.08)
kalman_history = kalman_tracker(observations, n_outcomes=n_outcomes, process_var=0.01, observation_var=0.1)

rmse_ewma = np.sqrt(((ewma_history - true_beta) ** 2).mean(axis=1))
rmse_kalman = np.sqrt(((kalman_history - true_beta) ** 2).mean(axis=1))
rmse = pd.DataFrame({
    'step': np.arange(n_steps),
    'EWMA': rmse_ewma,
    'Kalman': rmse_kalman,
})

## Visual comparison

We focus on outcome 0 (the sinusoidally perturbed channel) and aggregate error metrics.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
axes[0].plot(true_beta[:, 0], label='True β₀', color='black', linewidth=2)
axes[0].plot(ewma_history[:, 0], label='EWMA estimate', alpha=0.8)
axes[0].plot(kalman_history[:, 0], label='Kalman estimate', alpha=0.8)
axes[0].set_ylabel('β value')
axes[0].set_title('Outcome 0 drift vs. online estimates')
axes[0].legend()

axes[1].plot(rmse['step'], rmse['EWMA'], label='EWMA RMSE')
axes[1].plot(rmse['step'], rmse['Kalman'], label='Kalman RMSE')
axes[1].set_xlabel('Step')
axes[1].set_ylabel('RMSE')
axes[1].set_title('Rolling RMSE across the horizon')
axes[1].legend()
plt.tight_layout()
plt.show()

## Rolling error analysis

RMSE windows reveal how quickly each estimator recovers after sharp changes.

In [ ]:
window = 30
rolling_rmse = rmse.set_index('step').rolling(window=window, min_periods=1).mean()
rolling_rmse.plot(figsize=(10, 4))
plt.title(f'{window}-step rolling RMSE (lower is better)')
plt.ylabel('RMSE')
plt.xlabel('Step')
plt.tight_layout()
plt.show()

## Takeaways

* The Kalman tracker responds faster to shocks but can overshoot on quiet stretches.
* The EWMA estimator is smoother and lower variance once the drift stabilizes.
* Both trackers honor the WAMECU manifesto by adapting to *Weight–Atmosphere–Mechanics* perturbations without leaking exploitative strategies.